## ДЗ2 часть 1
### Такташева Катя, БКЛ182

Импортируем нужные модули:

In [41]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [42]:
import json
from pymystem3 import Mystem
m = Mystem()

### Задание 1: Текст книги

Запишем текст книги в переменную text:

In [43]:
with open('book.txt', 'r', encoding='utf-8') as fid:
    text = fid.read()

### Задание 2: MyStem

In [44]:
%%time
lemmas_ms = m.analyze(text)

Преобразуем полученные данные в формат .json и запишем в файл 'lemmas_mystem.json':

In [45]:
with open('lemmas_mystem.json', 'w', encoding='utf-8') as fid:
    json.dump(lemmas_ms, fid)

### Задание 3: PyMorphy
Импортируем нужные модули:

In [46]:
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Преобразуем текст в токены для анализа:

In [47]:
tokens = word_tokenize(text)
lemmas = []

In [48]:
%%time
for token in tokens:
    lemma = morph.parse(token)
    lemmas.append(lemma)

"Вытащим" из полученных данных информацию о словоформе, а также всех вариантах разбора этой словоформы, и добавим в словарь лексему, часть речи и число каждого из них.

**P.S.:** Далее при создании словаря наткнулась на проблему, что некоторые варианты парсинга одного слова могут различаться буквой е/ё (например, один из вариантов разбора слова 'весны' - 'вёсны'), поэтому в ключе заменим все 'ё' на 'е'.

In [49]:
dic = dict()
lemmas_pm = []
for i in range(len(lemmas)):
    dic = dict()
    dic[lemmas[i][0].word.replace('ё', 'е')] = []
    for wordform in lemmas[i]:
        dic[wordform.word.replace('ё', 'е')].append({
            'normal_form': wordform.normal_form,
            'pos': wordform.tag.POS,
            'number': wordform.tag.number
            })
    lemmas_pm.append(dic)

Преобразуем полученные данные в формат .json и запишем в файл 'lemmas_pymorphy.json':

In [50]:
with open('lemmas_pymorphy.json', 'w', encoding='utf-8') as fid:
    json.dump(lemmas_pm, fid)

### Задание 4.1: Части речи
**NB:** Для анализа будут использованы данные, распарсенные Mystem, потому что они более точные. Стоп-слова не убираем.

Импортируем Counter:

In [51]:
from collections import Counter

Преобразуем данные из формата json в объект питона:

In [52]:
parced_data = []
for line in open('lemmas_mystem.json', encoding='utf-8'):
    parced_data.extend(json.loads(line))

Посчитаем количество каждой из частей речи в тексте:

In [53]:
pos_list = []
for line in parced_data:
    if 'analysis' in line.keys():
        if line['analysis'] != []:
            pos = line['analysis'][0]['gr']
            pos = pos.replace('=', ',').split(',')[0]
            pos_list.append(pos)
counter_pos = Counter(pos_list)

Создадим строку frequency, в которую запишем все нужные для вывода даные: 

In [54]:
frequency = 'POS:\tДоля слов:\n'
for pos, count in counter_pos.items():
    percent = count / len(pos_list)
    frequency += pos + '\t' + str(percent) + '\n'

#### Ответ:

In [55]:
print('Частотность слов по частям речи:', frequency, sep='\n\n')

### Задание 4.2: Топ-20 глаголов и наречий
Пройдемся по словарю и добавим в списки глаголов и наречий лексемы. Для деления заменим знак =, используемый для отсечения части речи и ее неизменяемых признаков.

In [56]:
adv_list = []
verb_list = []
for line in parced_data:
    if 'analysis' in line.keys():
        if line['analysis'] != []:
            pos = line['analysis'][0]['gr'].replace('=', ',').split(',')[0]
            if pos == 'V':
                verb_list.append(line['analysis'][0]['lex'])
            elif pos == 'ADV':
                adv_list.append(line['analysis'][0]['lex'])

Создадим строку avd_top, в которую запишем топ-20 наречий, встречающихся в тексте: 

In [57]:
adv_top = '№:\tЛексема:\tКол-во употреблений:\n'
i = 1
for word, count in Counter(adv_list).most_common(20):
    adv_top += str(i) + '\t' + word + '\t\t' + str(count) + '\n'
    i += 1

Аналогично для глаголов:

In [58]:
verb_top = '№:\tЛексема:\tКол-во употреблений:\n'
i = 1
for word, count in Counter(verb_list).most_common(20):
    verb_top += str(i) + '\t' + word + '\t\t' + str(count) + '\n'
    i += 1

#### Ответ:

In [59]:
print('Топ-20 глаголов:', verb_top, sep='\n\n', end='\n')
print('Топ-20 наречий:', adv_top, sep='\n\n', end='\n')

### Задание 5: Топ-20 биграмм и триграмм
Создадим список лемм из текста:

In [60]:
words_lemmas = []
for word in parced_data:
    if 'analysis' in word.keys():
        if word['analysis'] != []:
            lemma = word['analysis'][0]['lex']
            words_lemmas.append(lemma)

Найдем все 2- и 3-граммы, объединим их в одном списке и создадим частотный словарь:

In [61]:
bigr = list(nltk.bigrams(words_lemmas))
trigr = list(nltk.trigrams(words_lemmas))
bigrams_counter = Counter(bigr)

Запишем топ-25 2-грамм в строку bigrams_top. Аналогично с 3-граммами.

In [62]:
bigrams_top = '№:\tКол-во:\tn-грамма:\n'
i = 1
for gram, count in bigrams_counter.most_common(25):
    bigrams_top += str(i) + '\t' + str(count) + '\t' + ' '.join(gram) + '\n'
    i += 1

In [63]:
trigrams_top = '№:\tКол-во:\tn-грамма:\n'
i = 1
for gram, count in Counter(trigr).most_common(25):
    trigrams_top += str(i) + '\t' + str(count) + '\t' + ' '.join(gram) + '\n'
    i += 1

#### Ответ:

In [64]:
print('Самые частотные 2-граммы:', bigrams_top, sep='\n\n')

При анализе 2-грамм можно заметить, что чамыми частотными являются имена героев произведения, а также фразы в принципе частотные для русского языка связки, типа "то есть", "так и", "и не" и т.д.

In [65]:
print('Самые частотные 3-граммы:', trigrams_top, sep='\n\n')

Получим схожие результаты: в топе находятся имя главного героя и, соответсвенно, его самые частотные действия: спрашивать, быть, увидеть и т.д., а также частые русские триграммы: время от времени, во всяком случае, в то что, и так далее. 